# OpenAI with PowerShell

### Overview

* What is OpenAI? ChatGPT?
* OpenAI vs. ChatGPT / InstructGPT
  * LLM = Large Language Model (2019, 2021)
  * AGI = Artificial General Intelligence
  * Subscription / Pricing / Tokens / Orgs / Budgets
  * Other LLMs: Google Bard
* REST API
  * GPT-3, Codex, DALL-E
  * Models: ada, babbage, curie, davinci
  * Completions
  * Moderations
  * Edits
  * Fine-Tuning
  * Embeddings
  * more...
* Prompts and Prompt Engineering
  * Phrasing / Ordering
* Thanks to Doug Finke for his [PowerShellAI](https://www.powershellgallery.com/packages/PowerShellAI/) module!

#### Example A

_How do I use MS Graph?_

#### Example B

_Show me how to connect to MS Graph with PowerShell_

#### Example C

_Without explaining, write the PowerShell code to connect to Microsoft Graph and query all Azure AD user accounts_

### Preparation

* Create an OpenAI Account
  * Read Pricing information: https://openai.com/api/pricing/
  * Review and try examples: https://platform.openai.com/examples
  * Create an API key (copy to clipboard)
* Store API key in a safe place (key vault, etc.)
* Import into environment variable "OpenAIKey" ($env:OpenAIKey)
* Install-Module PowerShellAI

### Using the API

Using CURL

In [ ]:
curl https://api.openai.com/v1/completions `
	-H 'Content-Type: application/json' `
	-H "Authorization: Bearer $($env:OpenAIKey)" `
	-d '{
		"model": "text-davinci-003",
		"prompt": "List the 3 most populated US states with population, and abbreviation, output as a markdown table",
		"max_tokens": 1000,
		"temperature": 0
	}'

Using Python 3

In [ ]:
# python
import os
import openai
openai.api_key = open('pykey.txt').read()
# openai.api_key = os.getenv('OpenAIKey')

question = "List the 3 most populated US states with population, and abbreviation, output in json format"
aimodel = "text-davinci-003"

os.system('cls')
print('Submitting request to openai...')
response = openai.Completion.create(model=aimodel,prompt=question,temperature=0,max_tokens=1000)
print(response.choices[0].text)

Invoke Script (Example)

In [ ]:
# python
import os
os.chdir('C:\\git\\powershell-labs')
os.system('cls')
exec(open('openaisample.py').read())

Using PowerShell

In [ ]:
$headers = @{
	'Content-Type'='application/json';
	'Authorization'="Bearer $($env:OpenAIKey)"
}

$body = @{
	"model" = "text-davinci-003"
	"prompt" = "List the 3 most populated US states with population, and abbreviation, output in XML"
	"max_tokens" = 1000
	"temperature" = 0
} | ConvertTo-Json

Invoke-RestMethod -Uri 'https://api.openai.com/v1/completions' -Headers $headers -Method POST -Body $body

In [ ]:
$response = Invoke-RestMethod -Uri 'https://api.openai.com/v1/completions' -Headers $headers -Method POST -Body $body
$response.choices | % { $_.text }

In [ ]:
$body = @{
	"model" = "text-curie-001"
	"prompt" = "Translate to Spanish: I really love my job!"
	"max_tokens" = 10
	"temperature" = 0
} | ConvertTo-Json
$response = Invoke-RestMethod -Uri 'https://api.openai.com/v1/completions' -Headers $headers -Method POST -Body $body
$response.choices | % { $_.text }

### PowerShellAI

* PowerShell Module written by Doug Finke (author of "ImportExcel") YouTube/GitHub: @DougFinke
* Provides a similar capability to the Python wrapper for the OpenAI API
* Install-Module PowerShellAI
* Uses an environment variable to store the API key ("OpenAIKey")

#### Completions

In [ ]:
Get-GPT3Completion -Prompt "If it is 9:45 PM in Virginia right now, what time is it in Seattle?"

In [ ]:
ai "If John is 42 years old, Bob is 40 years old, and Ray is older than Bob but younger than John, how old is Ray?"

In [ ]:
$text = @"
Write the comment help for the following powershell function:

function Get-DeptName {
	param (
		[parameter(Mandatory)][int]$DeptID,
		[parameter()][string]$SqlInstance = "sql001.contoso.local",
		[parameter()][string]$Database = "orginfo"
	)
	Invoke-DbaQuery -SqlInstance $SqlInstance -Database $Database -Query "select name from dept where id=$DeptID"
}
"@

ai $text

Leverage PowerShellAI and ImportExcel modules

In [ ]:
ai "last 5 states to join the nation, with their abbreviations and the year they joined, as a markdown table" |
	ConvertFrom-GPTMarkdownTable | Export-Excel -Show

Get commit description by asking OpenAI to analyze the git changes

In [ ]:
cd "c:\git\powershell-labs"
git stage --all
git status | ai "write me a detailed commit message for the staged changes in this repo"

Stage and Commit changes with AI-generated commit message

In [ ]:
cd "c:\git\azurelabs"
git stage --all
git status | ai "write me a detailed commit message for the staged changes in this repo" | % { git commit -m "$($_)" }
git push

#### Moderations

* Definition: Given a input text, outputs if the model classifies it as violating OpenAI's content policy.
* 7 deadly categories: sexual, hate, violence, self-harm, sexual/minors, hate/threatening, violence/graphic
* Supported models: text-moderation-stable, text-moderation-latest
* Reference: https://platform.openai.com/docs/api-reference/moderations

In [ ]:
Get-OpenAIModeration "I hate McAfee antivirus"

In [ ]:
Get-OpenAIModeration "I want to destroy McAfee antivirus." | foreach {
	$_.psobject.properties | where {$_.Value -eq $true} | select name
}

In [ ]:
Get-OpenAIModeration "I want to destroy them." | foreach {
	$_.psobject.properties | where {$_.Value -eq $true} | select name
}

In [ ]:
Get-OpenAIModeration "I want to smash it with a hammer!" | foreach {
	$_.psobject.properties | where {$_.Value -eq $true} | select name
}

In [ ]:
Get-OpenAIModeration "I want to smash them with a hammer!" | foreach {
	$_.psobject.properties | where {$_.Value -eq $true} | select name
}

In [ ]:
Get-OpenAIModeration "I want to destroy them." | foreach { $_.psobject.properties | where {$_.Value -eq $true} | select name }

#### Edits

* Definition: Given a prompt and an instruction, the model will return an edited version of the prompt
* Supported models: text-davinci-edit-001 or code-davinci-edit-001
* Reference: https://platform.openai.com/docs/api-reference/edits

In [ ]:
Get-OpenAIEdit "Nams of first thre presidents of the US" -Instruction "fix the spelling mistakes"

### More

In [ ]:
gpt "write a powershell function to return the intersection elements of two arrays"

In [ ]:
ai "last 5 states to join the nation, with their abbreviations and the year they joined"

In [ ]:
ai "last 5 states to join the nation, with their abbreviations and the year they joined, as a markdown table"

In [ ]:
ai "last 5 states to join the nation, with their abbreviations and the year they joined, as a markdown table" | ConvertFrom-GPTMarkdownTable

### Doug's Examples (NYPSUG)

In [ ]:
$p = '
For SQL create a customer table with the following columns:
id
First Name
Last Name
City
State
Postal
Country
WorkPhone
'
Get-GPT3Completion $p

In [ ]:
Get-GPT3Completion 'If cell A1 is less than 10 show "yes" otherwise show "no"'

In [ ]:
ai "write a function name 'FizzBuzz' that prints the numbers from 1 to 16, using PowerShell"